In [1]:
from datasets import load_dataset, load_metric, Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DefaultDataCollator
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm

# 1) Preprocessing

In [2]:
dataset = load_dataset("persiannlp/parsinlu_entailment")
dataset

Reusing dataset parsinlu_reading_comprehension (C:\Users\Danial\.cache\huggingface\datasets\persiannlp___parsinlu_reading_comprehension\parsinlu-repo\1.0.0\a99ad431d1637751f03fa9da3de66a41e93983d11116a0918a6efaaa72343d2d)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sent1', 'sent2', 'category', 'label'],
        num_rows: 755
    })
    test: Dataset({
        features: ['sent1', 'sent2', 'category', 'label'],
        num_rows: 1675
    })
    validation: Dataset({
        features: ['sent1', 'sent2', 'category', 'label'],
        num_rows: 270
    })
})

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(dataset["train"][0:10])

C:\Users\Danial\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,sent1,sent2,category,label
0,زنان به قدری بخش بزرگی از نیروی کار را تشکیل می دهند که به سختی می توان باور داشت که اگر این امر در مورد زنان صادق نباشد ، این امر می تواند صادق باشد.,مردان بخش عظیمی از نیروی کار هستند بنابراین تنها افراد مهم هستند.,translation-train,c
1,سالها است که کنگره در تلاش است تا اثربخشی مدیریت اطلاعات و فناوری را در دولت فدرال افزایش دهد.,کنگره بودجه ویژه ای برای مدیریت اطلاعات و فناوری در دولت فدرال دارد.,translation-train,n
2,سرامیک‌های زیست خنثی پس از قرارگیری در بدن میزبان خواص فیزیکی و مکانیکی خود را حفظ می‌کند.,خواص فیزیکی سرامیک‌ها قابل اندازه گیری است.,natural-wiki,n
3,دولت از هیچ قانونی که منجر به کاهش چشمگیر توانایی کشورمان در استفاده از زغال سنگ به عنوان منبع اصلی برق فعلی و آینده شود ، پشتیبانی نخواهد کرد.,قانونی که باعث کاهش استفاده از زغال سنگ به عنوان منبع انرژی شود ، توسط دولت پشتیبانی نمی شود.,translation-train,e
4,روش‌ها و الگوریتم‌های بهینه‌سازی به دو دسته الگوریتمهای دقیق (exact) و الگوریتم‌های تقریبی (approximate algorithms) تقسیم‌بندی می‌شوند.,آمار در دروس مدیریتی نقش مهمی را بازی میکند.,natural-wiki,n
5,گرچه بسیاری از اختلافات یاد شده در بین فقها برسربحث ستر صلاتی بوده است، اما اختلافات آنها در بحث کمّیت پوشش برای نماز به بحث کمّیت پوشش از نگاه نیز سرایت کرده است.,در داشتن حجاب به هنگام نماز در یک اتاق خالی و بدون وجود نامحرم چه فلسفه ای نهفته است؟,natural-wiki,n
6,"تیقیلر، کچارلی (به لاتین: Tığıllar, Koçarlı) یک منطقهٔ مسکونی در ترکیه است که در Koçarlı واقع شده‌است.",کوچه لره سو سپمیشم یک اهنگ ترکی قدیمی می باشد.,natural-wiki,n
7,وزیر کشور عراق در ادامه با بیان اینکه ایران رنج‌های زیادی بابت تحریم‌های اقتصادی مواجه شد اظهار کرد: اما ایران رهبر فرزانه و حکیم دارد که توانست از همه مشکلات عبور کند.,به گفته وزیر کشور عراق ایران توانست مشکلات تحریم را پشت سر بگذارد.,natural-miras,e
8,لینکلن چشم دوخت.,مرد چشمک زد.,translation-dev,c
9,"""سیف"" کتب و مقالات بسیاری در زمینهٔ اقتصاد، سیاست و تاریخ مشروطه به رشتهٔ تحریر درآورده یاترجمه کرده ‌است.","فعالیتهای آقای ""سیف"" بیشتر معطوف بر نوشتار و ترجمه بود.",natural-wiki,e


In [4]:
for data_set in ["train", "validation", "test"]:
#     print(np.unique(dataset[data_set]["label"], return_counts=True))
    bad_idxs = [i for i in range(len(dataset[data_set])) if dataset[data_set][i]["label"] not in ["c", "e", "n"]]
    print(dataset[data_set][bad_idxs])
    print()
    
dataset = dataset.filter(lambda example: example['label'] in ["c", "e", "n"])
dataset

Loading cached processed dataset at C:\Users\Danial\.cache\huggingface\datasets\persiannlp___parsinlu_reading_comprehension\parsinlu-repo\1.0.0\a99ad431d1637751f03fa9da3de66a41e93983d11116a0918a6efaaa72343d2d\cache-ddbade114b5e3de9.arrow
Loading cached processed dataset at C:\Users\Danial\.cache\huggingface\datasets\persiannlp___parsinlu_reading_comprehension\parsinlu-repo\1.0.0\a99ad431d1637751f03fa9da3de66a41e93983d11116a0918a6efaaa72343d2d\cache-def594aad0f0a1fa.arrow
Loading cached processed dataset at C:\Users\Danial\.cache\huggingface\datasets\persiannlp___parsinlu_reading_comprehension\parsinlu-repo\1.0.0\a99ad431d1637751f03fa9da3de66a41e93983d11116a0918a6efaaa72343d2d\cache-e57186a74fccface.arrow


{'sent1': ['xx'], 'sent2': ['xx'], 'category': ['natural-miras'], 'label': ['xx']}

{'sent1': [], 'sent2': [], 'category': [], 'label': []}

{'sent1': ['آنها برنامه هایی را برای افزایش کارآیی و به کارگیری منابع به طور مؤثر تثبیت کردند', 'این هتل نوع جدیدی از هتل های فوق ستاره و کوچک را به نمایش می گذارد که فضای اداری عمومی را به افرادی که از راه دور کار میکنند, می فروشد.'], 'sent2': ['برنامه های افزایش بهره وری تلفیق شدند ، زیرا آنها عمیقاً به کارایی اهمیت می دهند.', 'افرادی که از راه دور کار میکنند  فضای اداری عمومی را خریداری می کند.'], 'category': ['translation-dev', 'translation-dev'], 'label': ['-', '-']}



DatasetDict({
    train: Dataset({
        features: ['sent1', 'sent2', 'category', 'label'],
        num_rows: 754
    })
    test: Dataset({
        features: ['sent1', 'sent2', 'category', 'label'],
        num_rows: 1673
    })
    validation: Dataset({
        features: ['sent1', 'sent2', 'category', 'label'],
        num_rows: 270
    })
})

# 2)

In [5]:
model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)
model

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

In [6]:
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

str_to_int = {"e": 0, "n": 1, "c": 2}
def tokenize_function(examples):
    tokenized_batch = tokenizer(examples["sent1"], examples["sent2"], truncation=True, max_length=128)
    tokenized_batch["label"] = [str_to_int[label] for label in examples["label"]]
    return tokenized_batch

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

Loading cached processed dataset at C:\Users\Danial\.cache\huggingface\datasets\persiannlp___parsinlu_reading_comprehension\parsinlu-repo\1.0.0\a99ad431d1637751f03fa9da3de66a41e93983d11116a0918a6efaaa72343d2d\cache-dd276b116c6d8910.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

Loading cached processed dataset at C:\Users\Danial\.cache\huggingface\datasets\persiannlp___parsinlu_reading_comprehension\parsinlu-repo\1.0.0\a99ad431d1637751f03fa9da3de66a41e93983d11116a0918a6efaaa72343d2d\cache-1fa8734e982e028f.arrow


DatasetDict({
    train: Dataset({
        features: ['sent1', 'sent2', 'category', 'label', 'input_ids', 'attention_mask'],
        num_rows: 754
    })
    test: Dataset({
        features: ['sent1', 'sent2', 'category', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1673
    })
    validation: Dataset({
        features: ['sent1', 'sent2', 'category', 'label', 'input_ids', 'attention_mask'],
        num_rows: 270
    })
})

In [7]:
training_args = TrainingArguments(
    output_dir="q1_xlm-r", 
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    logging_steps = 20,
    learning_rate=2e-5,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    load_best_model_at_end=True,
    weight_decay=0.001,
    save_total_limit = 1,
    metric_for_best_model="accuracy",
    group_by_length = True,
    seed=0
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: category, sent2, sent1. If category, sent2, sent1 are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Danial\Anaconda3\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 754
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 480


Epoch,Training Loss,Validation Loss,Accuracy
1,1.103400,1.096694,0.361626
2,1.093100,1.104991,0.364614
3,1.081600,1.099134,0.372385
4,1.060300,1.069397,0.435744
5,0.979000,1.147930,0.377764
6,0.910000,1.078549,0.451883
7,0.746700,1.141735,0.468022
8,0.606100,1.203782,0.472206
9,0.487000,1.314937,0.467424
10,0.481900,1.270491,0.471608


The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: category, sent2, sent1. If category, sent2, sent1 are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1673
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: category, sent2, sent1. If category, sent2, sent1 are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1673
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: category, sent2, sent1. If category, sent2, sent1 are not expected by `XLMRobertaForSequenc

TrainOutput(global_step=480, training_loss=0.8695441822210948, metrics={'train_runtime': 227.0969, 'train_samples_per_second': 33.202, 'train_steps_per_second': 2.114, 'total_flos': 224025658393572.0, 'train_loss': 0.8695441822210948, 'epoch': 10.0})

In [8]:
trainer.evaluate(tokenized_datasets["test"])

The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: category, sent2, sent1. If category, sent2, sent1 are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1673
  Batch size = 16


{'eval_loss': 1.270491361618042,
 'eval_accuracy': 0.47160789001793185,
 'eval_runtime': 8.8449,
 'eval_samples_per_second': 189.149,
 'eval_steps_per_second': 11.871,
 'epoch': 10.0}

# 3)

In [28]:
model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)

metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

str_to_int = {"e": 0, "n": 1, "c": 2}
def tokenize_function(examples):
    tokenized_batch = tokenizer(examples["sent1"], examples["sent2"], truncation=True)
    tokenized_batch["label"] = [str_to_int[label] for label in examples["label"]]
    return tokenized_batch

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/HooshvareLab/bert-base-parsbert-uncased/resolve/main/config.json from cache at C:\Users\Danial/.cache\huggingface\transformers\d3b7c3283a6a4ad4471f59269c9de8adadfab0b05eebf49a64e046fca56cdab2.58cfea678e7bd2c1de3bfd4a5357101526b9fbc32a994b9456047e55b0afbebe
Model config BertConfig {
  "_name_or_path": "HooshvareLab/bert-base-parsbert-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2

DatasetDict({
    train: Dataset({
        features: ['sent1', 'sent2', 'category', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 754
    })
    test: Dataset({
        features: ['sent1', 'sent2', 'category', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1673
    })
    validation: Dataset({
        features: ['sent1', 'sent2', 'category', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 270
    })
})

In [31]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [29]:
training_args = TrainingArguments(
    output_dir="q1_parsbert", 
    evaluation_strategy="epoch",
    logging_steps = 20,
    learning_rate=5e-5,
    num_train_epochs=3,
    warmup_steps=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy = "epoch",
    load_best_model_at_end=True,
    save_total_limit = 1,
    metric_for_best_model="accuracy",
    group_by_length = True,
    seed=0
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: category, sent1, sent2. If category, sent1, sent2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Danial\Anaconda3\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 754
  Num Epochs = 3
  Instantaneous batch size per 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.109000,1.035449,0.468619
2,0.730800,1.148304,0.496712
3,0.452300,1.253495,0.512253


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: category, sent1, sent2. If category, sent1, sent2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1673
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: category, sent1, sent2. If category, sent1, sent2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1673
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: category, sent1, sent2. If category, sent1, sent2 are not expected by `BertForSequenceClassification.forward`,  you can saf

TrainOutput(global_step=285, training_loss=0.8127144253044798, metrics={'train_runtime': 76.4352, 'train_samples_per_second': 29.594, 'train_steps_per_second': 3.729, 'total_flos': 87821297674236.0, 'train_loss': 0.8127144253044798, 'epoch': 3.0})

In [30]:
trainer.evaluate(tokenized_datasets["test"])

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: category, sent1, sent2. If category, sent1, sent2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1673
  Batch size = 8


{'eval_loss': 1.253495216369629,
 'eval_accuracy': 0.5122534369396294,
 'eval_runtime': 7.6328,
 'eval_samples_per_second': 219.186,
 'eval_steps_per_second': 27.513,
 'epoch': 3.0}